In [ ]:
# very positive 이상의 게임만 추출해서 봤는데 9개 밖에 되지 않음.

import os
import requests
import pandas as pd
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed

# 1. 환경 변수 로드 (필요 시)
load_dotenv()

# 2. JSON 파일에서 게임 리스트 불러오기 (app_id와 name 모두 불러오기)
game_list_file = "steamdb_indie_moba_merged.json"
df_games = pd.read_json(game_list_file, orient="records", encoding="utf-8")
# app_id를 문자열로 변환
df_games["app_id"] = df_games["app_id"].astype(str)
# app_id와 게임 이름 매핑 딕셔너리 생성
app_id_to_name = df_games.set_index("app_id")["name"].to_dict()
app_ids = df_games["app_id"].tolist()
print(f"총 {len(app_ids)}개의 게임 ID를 불러왔습니다.")
print("예시 app_id:", app_ids[:5])

# 3. Steam API를 사용하여 각 게임의 review_score_desc 값을 가져오는 함수 (게임 이름 포함)
def fetch_game_review_summary(app_id):
    """
    주어진 app_id에 대해 Steam API를 호출해 review_score_desc 값을 가져옵니다.
    반환: (app_id, name, review_score_desc) 튜플
    """
    api_url = f"https://store.steampowered.com/appreviews/{app_id}?json=1"
    try:
        response = requests.get(api_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            query_summary = data.get("query_summary", {})
            review_desc = query_summary.get("review_score_desc", "").lower()
            # 게임 이름은 매핑에서 가져옴 (없으면 빈 문자열)
            game_name = app_id_to_name.get(app_id, "")
            return app_id, game_name, review_desc
        else:
            print(f"[ERROR] 앱 ID {app_id}: HTTP {response.status_code} 에러 발생")
            return app_id, app_id_to_name.get(app_id, ""), None
    except Exception as e:
        print(f"[EXCEPTION] 앱 ID {app_id} 호출 중 예외 발생: {e}")
        return app_id, app_id_to_name.get(app_id, ""), None

# 4. "Very Positive" 이상인 게임 필터링 (병렬 처리)
positive_reviews = {"very positive", "overwhelmingly positive"}
filtered_games = []

with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_appid = {executor.submit(fetch_game_review_summary, app_id): app_id for app_id in app_ids}
    for future in as_completed(future_to_appid):
        app_id, game_name, review_desc = future.result()
        if review_desc in positive_reviews:
            filtered_games.append({"app_id": app_id, "name": game_name})

# 5. 결과 확인 및 JSON 파일로 저장
print(f"\n'Very Positive' 이상 게임 수: {len(filtered_games)}")
print("예시:", filtered_games[:5])

filtered_games_df = pd.DataFrame(filtered_games)
filtered_games_df.to_json("filtered_very_positive_games.json", orient="records", force_ascii=False, indent=4)
print("✅ 'filtered_very_positive_games.json' 파일로 저장 완료.")


총 206개의 게임 ID를 불러왔습니다.
예시 app_id: ['1310990', '572220', '360620', '1777280', '204300']

'Very Positive' 이상 게임 수: 9
예시: [{'app_id': '204300', 'name': 'Awesomenauts - the 2D moba'}, {'app_id': '572220', 'name': 'MageQuit'}, {'app_id': '360620', 'name': 'Spellsworn'}, {'app_id': '428460', 'name': 'Kings of Israel'}, {'app_id': '605920', 'name': 'VOLTED'}]
✅ 'filtered_very_positive_games.json' 파일로 저장 완료.


In [2]:
import os
import requests
import pandas as pd
import time
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed

# 1. 환경 변수 로드 (필요한 경우)
load_dotenv()

# 2. "Very Positive" 게임 리스트 파일에서 app_id 추출
game_list_file = "filtered_very_positive_games.json"
try:
    df_filtered_games = pd.read_json(game_list_file, orient="records", encoding="utf-8")
    app_ids = df_filtered_games["app_id"].astype(str).tolist()
    print(f"총 {len(app_ids)}개의 'Very Positive' 게임 ID를 불러왔습니다.")
    print("예시 app_id:", app_ids[:5])
except Exception as e:
    print(f"게임 리스트 파일 로드 중 오류 발생: {e}")
    app_ids = []

# 3. Steam 리뷰 API를 호출하여 모든 정보를 가져오는 함수
def fetch_api_data(app_id, num_reviews=20, language="english"):
    """
    주어진 app_id에 대해 Steam 리뷰 API를 호출하여, 반환되는 모든 정보를 가져옵니다.
    반환값은 app_id와 함께 전체 API 응답을 포함하는 dict입니다.
    """
    api_url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&num_per_page={num_reviews}&language={language}"
    try:
        response = requests.get(api_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            return {"app_id": app_id, **data}
        else:
            print(f"[ERROR] 앱 ID {app_id}: HTTP {response.status_code} 에러 발생")
            return {"app_id": app_id, "error": response.status_code}
    except Exception as e:
        print(f"[EXCEPTION] 앱 ID {app_id} 호출 중 예외 발생: {e}")
        return {"app_id": app_id, "exception": str(e)}

# 4. 여러 app_id에 대해 병렬로 API 호출하는 함수
def fetch_all_api_data(app_ids, num_reviews=20, language="english", max_workers=10):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # futures에 각 app_id에 대한 API 호출 작업을 제출
        futures = {executor.submit(fetch_api_data, app_id, num_reviews, language): app_id for app_id in app_ids}
        for future in as_completed(futures):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"[EXCEPTION] 작업 처리 중 오류 발생: {e}")
    return results

# 5. 모든 대상 게임에 대해 API 데이터 수집 (병렬 처리)
all_api_data = fetch_all_api_data(app_ids, num_reviews=20, language="english", max_workers=10)

# 6. 중첩된 JSON 응답을 평면화하여 DataFrame 생성
df_api = pd.json_normalize(all_api_data)
print("API 응답 데이터 플랫닝 후 컬럼들:")
print(df_api.columns)

# 7. 모든 API 응답 데이터를 JSON 파일로 저장 (추후 컬럼 선택 및 전처리 참고)
output_file = "all_api_data.json"
try:
    df_api.to_json(output_file, orient="records", force_ascii=False, indent=4)
    print(f"✅ '{output_file}' 파일로 저장 완료.")
except Exception as e:
    print(f"JSON 저장 중 오류 발생: {e}")


총 9개의 'Very Positive' 게임 ID를 불러왔습니다.
예시 app_id: ['572220', '204300', '360620', '428460', '605920']
API 응답 데이터 플랫닝 후 컬럼들:
Index(['app_id', 'success', 'reviews', 'cursor', 'query_summary.num_reviews',
       'query_summary.review_score', 'query_summary.review_score_desc',
       'query_summary.total_positive', 'query_summary.total_negative',
       'query_summary.total_reviews'],
      dtype='object')
✅ 'all_api_data.json' 파일로 저장 완료.


In [7]:
import os
import requests
import pandas as pd
import time
from datetime import datetime
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

# 1. 환경 변수 로드
load_dotenv()

# 2. "Very Positive" 게임 리스트 파일에서 게임 이름과 app_id 불러오기
filtered_games_file = "filtered_very_positive_games.json"
try:
    df_filtered_games = pd.read_json(filtered_games_file, orient="records", encoding="utf-8")
    # app_id를 정수형으로 변환 후, 게임 이름 매핑 딕셔너리 생성
    df_filtered_games['app_id'] = df_filtered_games['app_id'].astype(int)
    app_id_to_name = df_filtered_games.set_index("app_id")["name"].to_dict()
    # app_id 리스트를 문자열 형태로 저장 (API URL에 사용)
    app_ids = df_filtered_games["app_id"].astype(str).tolist()
    print(f"총 {len(app_ids)}개의 'Very Positive' 게임 ID를 불러왔습니다.")
    print("예시 app_id:", app_ids[:5])
except Exception as e:
    print(f"게임 리스트 파일 로드 중 오류 발생: {e}")
    app_ids = []
    app_id_to_name = {}

# 3. Steam 리뷰 API를 호출하여 모든 정보를 가져오는 함수 (게임 이름 포함)
def fetch_api_data(app_id, num_reviews=20, language="english"):
    """
    주어진 app_id에 대해 Steam 리뷰 API를 호출하여, 반환되는 모든 정보를 가져옵니다.
    미리 준비한 app_id_to_name 매핑을 통해 'name' 키로 게임 이름을 추가합니다.
    반환값은 app_id, name과 전체 API 응답을 포함하는 dict입니다.
    """
    api_url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&num_per_page={num_reviews}&language={language}"
    try:
        response = requests.get(api_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            # 게임 이름 추가 (매핑에서 찾을 수 없는 경우 빈 문자열)
            game_name = app_id_to_name.get(int(app_id), "")
            return {"app_id": app_id, "name": game_name, **data}
        else:
            print(f"[ERROR] 앱 ID {app_id}: HTTP {response.status_code} 에러 발생")
            return {"app_id": app_id, "name": app_id_to_name.get(int(app_id), ""), "error": response.status_code}
    except Exception as e:
        print(f"[EXCEPTION] 앱 ID {app_id} 호출 중 예외 발생: {e}")
        return {"app_id": app_id, "name": app_id_to_name.get(int(app_id), ""), "exception": str(e)}

# 4. 여러 app_id에 대해 병렬로 API 호출하는 함수
def fetch_all_api_data(app_ids, num_reviews=20, language="english", max_workers=10):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_api_data, app_id, num_reviews, language): app_id for app_id in app_ids}
        for future in as_completed(futures):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                app_id = futures[future]
                print(f"[EXCEPTION] 앱 ID {app_id} 처리 중 오류 발생: {e}")
    return results

# 5. 모든 대상 게임에 대해 API 데이터 수집 (병렬 처리)
all_api_data = fetch_all_api_data(app_ids, num_reviews=20, language="english", max_workers=10)

# 6. 수집된 API 응답 데이터 평면화 (중첩 JSON 처리)
df_api = pd.json_normalize(all_api_data)
print("API 응답 데이터 평면화 후 컬럼들:")
print(df_api.columns)

# 7. 리뷰 데이터 전처리: 필요한 정보만 추출하여 리뷰 테이블 생성
processed_reviews = []
for index, row in df_api.iterrows():
    app_id = row.get("app_id")
    game_name = row.get("name")
    # 'reviews'가 리스트 형태로 있는 컬럼 평면화
    reviews = row.get("reviews", [])
    if not isinstance(reviews, list):
        continue
    for review in reviews:
        # 필수 리뷰 정보 추출 및 클렌징
        review_id = review.get("recommendationid")
        review_text = review.get("review", "").strip()
        timestamp = review.get("timestamp_created")
        if timestamp:
            review_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        else:
            review_time = None
        playtime = review.get("author", {}).get("playtime_forever", 0)
        sentiment = "Positive" if review.get("voted_up") else "Negative"
        # 영어 리뷰만 유지 (API 호출 시 영어 필터 적용했으므로 대부분 영어일 것으로 예상)
        if review.get("language") != "english" or not review_text:
            continue
        processed_reviews.append({
            "app_id": app_id,
            "name": game_name,
            "review_id": review_id,
            "timestamp": review_time,
            "playtime_forever": playtime,
            "review_text": review_text,
            "sentiment": sentiment
        })

# 8. 최종 전처리된 리뷰 데이터를 DataFrame으로 변환
df_reviews = pd.DataFrame(processed_reviews)
print("=== 최종 전처리된 리뷰 데이터 (샘플) ===")
print(df_reviews.head())
print(df_reviews.info())

# 9. 전처리된 데이터를 JSON 파일로 저장 (UTF-8, 인덴트 4)
output_file = "processed_reviews.json"
try:
    df_reviews.to_json(output_file, orient="records", force_ascii=False, indent=4)
    print(f"✅ '{output_file}' 파일로 저장 완료.")
except Exception as e:
    print(f"JSON 저장 중 오류 발생: {e}")


총 9개의 'Very Positive' 게임 ID를 불러왔습니다.
예시 app_id: ['204300', '572220', '360620', '428460', '605920']
API 응답 데이터 평면화 후 컬럼들:
Index(['app_id', 'name', 'success', 'reviews', 'cursor',
       'query_summary.num_reviews', 'query_summary.review_score',
       'query_summary.review_score_desc', 'query_summary.total_positive',
       'query_summary.total_negative', 'query_summary.total_reviews'],
      dtype='object')
=== 최종 전처리된 리뷰 데이터 (샘플) ===
   app_id           name  review_id            timestamp  playtime_forever  \
0  431250  Mushroom Wars  183163656  2024-12-20 03:45:03               644   
1  572220       MageQuit  189386962  2025-03-04 21:22:14              2247   
2  572220       MageQuit  188088894  2025-02-17 00:35:13               757   
3  572220       MageQuit  188235475  2025-02-19 00:24:42              1239   
4  572220       MageQuit  189557736  2025-03-07 02:31:19              3821   

                                         review_text sentiment  
0  I loved playing this gam

C:\Users\02105\AppData\Local\Temp\ipykernel_11888\2450493727.py:88: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  review_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')


In [ ]:
#인디모바 정보확인용

import os
import requests
import pandas as pd
import time
from datetime import datetime
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

# 1. 환경 변수 로드
load_dotenv()

# 2. 게임 리스트 JSON 파일에서 게임 이름과 app_id 불러오기
game_list_file = "steamdb_indie_moba_merged.json"
try:
    df_games = pd.read_json(game_list_file, orient="records", encoding="utf-8")
    # app_id를 문자열로 변환
    df_games["app_id"] = df_games["app_id"].astype(str)
    # app_id와 게임 이름 매핑 딕셔너리 생성
    app_id_to_name = df_games.set_index("app_id")["name"].to_dict()
    # 전체 app_id 리스트
    app_ids = df_games["app_id"].tolist()
    print(f"총 {len(app_ids)}개의 게임 ID를 불러왔습니다.")
    print("예시 app_id:", app_ids[:5])
except Exception as e:
    print(f"게임 리스트 파일 로드 중 오류 발생: {e}")
    app_ids = []
    app_id_to_name = {}

# 3. Steam API를 호출하여 각 게임의 review_score_desc를 가져오는 함수
def fetch_game_review_summary(app_id):
    """
    주어진 app_id에 대해 Steam API를 호출해 review_score_desc 값을 가져옵니다.
    반환: (app_id, name, review_score_desc) 튜플
    """
    api_url = f"https://store.steampowered.com/appreviews/{app_id}?json=1"
    try:
        response = requests.get(api_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            query_summary = data.get("query_summary", {})
            review_desc = query_summary.get("review_score_desc", "").lower()
            # 매핑 딕셔너리에서 게임 이름 가져오기 (없으면 빈 문자열)
            game_name = app_id_to_name.get(app_id, "")
            return app_id, game_name, review_desc
        else:
            print(f"[ERROR] 앱 ID {app_id}: HTTP {response.status_code} 에러 발생")
            return app_id, app_id_to_name.get(app_id, ""), None
    except Exception as e:
        print(f"[EXCEPTION] 앱 ID {app_id} 호출 중 예외 발생: {e}")
        return app_id, app_id_to_name.get(app_id, ""), None

# 4. 여러 app_id에 대해 병렬로 API 호출하는 함수
def fetch_all_review_summaries(app_ids, max_workers=10):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_game_review_summary, app_id): app_id for app_id in app_ids}
        for future in as_completed(futures):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                app_id = futures[future]
                print(f"[EXCEPTION] 앱 ID {app_id} 처리 중 오류 발생: {e}")
    return results

# 5. 모든 대상 게임에 대해 review summary 수집 (병렬 처리)
all_review_summaries = fetch_all_review_summaries(app_ids, max_workers=10)

# 6. 필터링 모드 설정
# FILTER_MODE 옵션: "all" (전체), "vp_only" (Very Positive 이상만), "non_vp" (Very Positive 이상이 아닌 게임)
FILTER_MODE = "all"  # 필요에 따라 "vp_only" 또는 "non_vp"로 변경

# 긍정 리뷰 조건
positive_reviews = {"very positive", "overwhelmingly positive"}
filtered_games = []

# 7. 조건에 맞게 게임 필터링
for app_id, game_name, review_desc in all_review_summaries:
    if FILTER_MODE == "all":
        filtered_games.append({"app_id": app_id, "name": game_name, "review_score_desc": review_desc})
    elif FILTER_MODE == "vp_only" and review_desc in positive_reviews:
        filtered_games.append({"app_id": app_id, "name": game_name, "review_score_desc": review_desc})
    elif FILTER_MODE == "non_vp" and review_desc not in positive_reviews:
        filtered_games.append({"app_id": app_id, "name": game_name, "review_score_desc": review_desc})

# 8. 결과 확인 및 JSON 파일로 저장
print(f"\n필터 모드 '{FILTER_MODE}'에 해당하는 게임 수: {len(filtered_games)}")
print("예시:", filtered_games[:5])
filtered_games_df = pd.DataFrame(filtered_games)
output_summary_file = "filtered_games_summary.json"
try:
    filtered_games_df.to_json(output_summary_file, orient="records", force_ascii=False, indent=4)
    print(f"✅ '{output_summary_file}' 파일로 저장 완료.")
except Exception as e:
    print(f"JSON 저장 중 오류 발생: {e}")


총 206개의 게임 ID를 불러왔습니다.
예시 app_id: ['1310990', '572220', '360620', '1777280', '204300']

필터 모드 'all'에 해당하는 게임 수: 206
예시: [{'app_id': '572220', 'name': 'MageQuit', 'review_score_desc': 'very positive'}, {'app_id': '206500', 'name': 'AirMech', 'review_score_desc': 'mostly positive'}, {'app_id': '204300', 'name': 'Awesomenauts - the 2D moba', 'review_score_desc': 'very positive'}, {'app_id': '534780', 'name': 'XGun-Weapon Evolution', 'review_score_desc': '7 user reviews'}, {'app_id': '3027820', 'name': 'Nexus Rumble: The Ultimate Showdown', 'review_score_desc': '2 user reviews'}]
✅ 'filtered_games_summary.json' 파일로 저장 완료.


In [9]:
# 인디모다 전처리한 전체 리뷰 확인

import os
import requests
import pandas as pd
import time
import re
from datetime import datetime
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

# 1. 환경 변수 로드
load_dotenv()

# 2. "Very Positive" 게임 리스트 파일에서 게임 이름과 app_id 불러오기
game_list_file = "filtered_very_positive_games.json"
try:
    df_filtered_games = pd.read_json(game_list_file, orient="records", encoding="utf-8")
    # app_id를 정수형으로 변환 후, 게임 이름 매핑 딕셔너리 생성
    df_filtered_games['app_id'] = df_filtered_games['app_id'].astype(int)
    app_id_to_name = df_filtered_games.set_index("app_id")["name"].to_dict()
    # app_id 리스트를 문자열 형태로 저장 (API URL에 사용)
    app_ids = df_filtered_games["app_id"].astype(str).tolist()
    print(f"총 {len(app_ids)}개의 'Very Positive' 게임 ID를 불러왔습니다.")
    print("예시 app_id:", app_ids[:5])
except Exception as e:
    print(f"게임 리스트 파일 로드 중 오류 발생: {e}")
    app_ids = []
    app_id_to_name = {}

# 3. 리뷰 텍스트 전처리 함수
def clean_review_text(text):
    """
    리뷰 텍스트를 소문자화하고, 알파벳, 숫자, 공백을 제외한 모든 특수 문자를 제거하며,
    중복 공백을 단일 공백으로 변환합니다.
    """
    text = text.lower()  # 소문자 변환
    text = re.sub(r"[^a-z0-9\s]", " ", text)  # 알파벳, 숫자, 공백 외 제거
    text = re.sub(r"\s+", " ", text).strip()  # 중복 공백 제거 및 앞뒤 공백 제거
    return text

# 4. Steam 리뷰 API를 호출하여 전체 리뷰 데이터를 가져오는 함수 (게임 이름 포함)
def fetch_all_reviews_for_app(app_id, language="english", delay=1):
    """
    주어진 app_id에 대해 Steam 리뷰 API를 사용하여 전체 리뷰를 가져옵니다.
    API는 cursor 기반 페이지네이션을 지원하므로, cursor가 반환되는 동안 반복 호출합니다.
    각 리뷰에서 게임 이름, review_id, timestamp, playtime, review_text, sentiment 정보를 추출합니다.
    """
    all_reviews = []
    cursor = "*"  # 초기 cursor 값은 "*"입니다.
    while True:
        api_url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&num_per_page=100&language={language}&cursor={cursor}"
        try:
            response = requests.get(api_url, timeout=10)
            if response.status_code != 200:
                print(f"[ERROR] 앱 ID {app_id}: HTTP {response.status_code} 에러 발생")
                break
            data = response.json()
            reviews = data.get("reviews", [])
            if not reviews:
                break
            for review in reviews:
                review_id = review.get("recommendationid")
                # 원래 리뷰 텍스트 전처리 적용
                raw_text = review.get("review", "").strip()
                review_text = clean_review_text(raw_text)
                timestamp = review.get("timestamp_created")
                if timestamp:
                    review_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
                else:
                    review_time = None
                playtime = review.get("author", {}).get("playtime_forever", 0)
                sentiment = "Positive" if review.get("voted_up") else "Negative"
                # 영어 리뷰만 유지 (API 호출 시 영어 필터 적용했으므로 대부분 영어일 것으로 예상)
                if review.get("language") != "english" or not review_text:
                    continue
                all_reviews.append({
                    "app_id": app_id,
                    "name": app_id_to_name.get(int(app_id), ""),
                    "review_id": review_id,
                    "timestamp": review_time,
                    "playtime_forever": playtime,
                    "review_text": review_text,
                    "sentiment": sentiment
                })
            # 다음 페이지 cursor 업데이트
            new_cursor = data.get("cursor")
            if not new_cursor or new_cursor == cursor:
                break
            cursor = new_cursor
            time.sleep(delay)
        except Exception as e:
            print(f"[EXCEPTION] 앱 ID {app_id} 리뷰 수집 중 예외 발생: {e}")
            break
    return all_reviews

# 5. 여러 app_id에 대해 병렬로 전체 리뷰 데이터 수집
def fetch_all_reviews(app_ids, language="english", max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_all_reviews_for_app, app_id, language): app_id for app_id in app_ids}
        for future in as_completed(futures):
            app_id = futures[future]
            try:
                reviews = future.result()
                results.extend(reviews)
                print(f"앱 ID {app_id}의 리뷰 {len(reviews)}건 수집 완료.")
            except Exception as e:
                print(f"[EXCEPTION] 앱 ID {app_id} 처리 중 오류 발생: {e}")
    return results

# 6. 모든 대상 게임에 대해 전체 리뷰 데이터 수집 (병렬 처리)
all_reviews_data = fetch_all_reviews(app_ids, language="english", max_workers=5)

# 7. 수집된 리뷰 데이터를 DataFrame으로 변환
df_reviews = pd.DataFrame(all_reviews_data)
print("=== 최종 전처리된 리뷰 데이터 (샘플) ===")
print(df_reviews.head())
print(df_reviews.info())

# 8. 전처리된 데이터를 JSON 파일로 저장 (UTF-8, 인덴트 4)
output_file = "processed_all_reviews.json"
try:
    df_reviews.to_json(output_file, orient="records", force_ascii=False, indent=4)
    print(f"✅ '{output_file}' 파일로 저장 완료.")
except Exception as e:
    print(f"JSON 저장 중 오류 발생: {e}")

# 9. (추가) app_id, name, review_text 컬럼만 추출하여 CSV 파일로 저장
csv_output_file = "review_texts.csv"
try:
    df_reviews[['app_id', 'name', 'review_text']].to_csv(csv_output_file, index=False, encoding="utf-8-sig")
    print(f"✅ '{csv_output_file}' 파일로 CSV 저장 완료.")
except Exception as e:
    print(f"CSV 저장 중 오류 발생: {e}")


총 9개의 'Very Positive' 게임 ID를 불러왔습니다.
예시 app_id: ['204300', '572220', '360620', '428460', '605920']


C:\Users\02105\AppData\Local\Temp\ipykernel_11888\2295014143.py:70: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  review_time = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')


앱 ID 428460의 리뷰 2건 수집 완료.
앱 ID 605920의 리뷰 3건 수집 완료.
앱 ID 360620의 리뷰 140건 수집 완료.
앱 ID 63200의 리뷰 1건 수집 완료.
앱 ID 204300의 리뷰 71건 수집 완료.
앱 ID 336420의 리뷰 7건 수집 완료.
앱 ID 558100의 리뷰 100건 수집 완료.
앱 ID 572220의 리뷰 99건 수집 완료.
앱 ID 431250의 리뷰 2건 수집 완료.
=== 최종 전처리된 리뷰 데이터 (샘플) ===
   app_id             name  review_id            timestamp  playtime_forever  \
0  428460  Kings of Israel  181973221  2024-12-05 18:14:39               292   
1  428460  Kings of Israel  181973221  2024-12-05 18:14:39               292   
2  605920           VOLTED  182479332  2024-12-11 08:09:54                49   
3  605920           VOLTED  186000812  2025-01-20 07:30:23               515   
4  605920           VOLTED  186000812  2025-01-20 07:30:23               515   

                                         review_text sentiment  
0                                                fun  Positive  
1                                                fun  Positive  
2  i have never played online for this game just ...  Pos

In [ ]:
# keyBERT로 키워드추출 
import re
import pandas as pd
from keybert import KeyBERT
import spacy
import nltk

# NLTK 리소스 다운로드 (최초 실행 시 한 번만)
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

# SpaCy 영어 모델 로드 (환경에 설치되어 있어야 함)
nlp = spacy.load("en_core_web_sm")
stop_words = set(nltk.corpus.stopwords.words("english"))

def clean_review_text(text):
    """
    리뷰 텍스트 전처리 함수:
    1. 소문자화
    2. 알파벳과 숫자, 공백 이외의 문자 제거
    3. SpaCy를 사용해 토큰화 및 표제어 추출, 불용어 및 구두점 제거
    4. 전처리된 토큰을 다시 하나의 문자열로 결합
    """
    text = text.lower()
    # 알파벳, 숫자, 공백 외 제거
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_stop and not token.is_punct and not token.is_space]
    return " ".join(tokens)

# KeyBERT 모델 로드
kw_model = KeyBERT()

# 1. CSV 파일 불러오기 (app_id, name, review_text 컬럼 포함)
csv_file = "review_texts.csv"
df = pd.read_csv(csv_file, encoding="utf-8-sig")
print("불러온 데이터 (샘플):")
print(df.head())

# 2. 리뷰 텍스트 전처리 적용
df["cleaned_text"] = df["review_text"].apply(clean_review_text)
print("\n전처리 전/후 비교:")
print(df[["review_text", "cleaned_text"]].head())

# 3. KeyBERT를 이용해 각 리뷰에서 키워드 추출 (예: 1~2그램 키워드 상위 5개)
def extract_keywords(text, top_n=5):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=top_n)
    return keywords

df["keywords"] = df["cleaned_text"].apply(lambda x: extract_keywords(x, top_n=5))
print("\n키워드 추출 결과 (샘플):")
print(df[["review_text", "cleaned_text", "keywords"]].head())

# 4. 결과 CSV 파일로 저장
output_csv = "review_keywords.csv"
df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"✅ 결과가 '{output_csv}' 파일로 저장되었습니다.")


c:\Users\02105\anaconda3\envs\T1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\02105\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\02105\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\02105\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
c:\Users\02105\anaconda3\envs\T1\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\02105\.cache\huggingface\hub\mode

불러온 데이터 (샘플):
   app_id             name                                        review_text
0  428460  Kings of Israel                                                fun
1  428460  Kings of Israel                                                fun
2  605920           VOLTED  i have never played online for this game just ...
3  605920           VOLTED                         drive car shoot and defend
4  605920           VOLTED                         drive car shoot and defend

전처리 전/후 비교:
                                         review_text  \
0                                                fun   
1                                                fun   
2  i have never played online for this game just ...   
3                         drive car shoot and defend   
4                         drive car shoot and defend   

                          cleaned_text  
0                                  fun  
1                                  fun  
2  play online game play bot fun chaos  
3   

In [8]:
import re
import pandas as pd
from keybert import KeyBERT
import spacy
import nltk

# NLTK 리소스 다운로드 (최초 실행 시 한 번만)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# SpaCy 영어 모델 로드 (환경에 설치되어 있어야 함)
nlp = spacy.load("en_core_web_sm")
stop_words = set(nltk.corpus.stopwords.words("english"))

def clean_review_text(text):
    """
    리뷰 텍스트 전처리 함수:
    1. 소문자화
    2. 알파벳, 숫자, 공백 이외의 문자 제거
    3. SpaCy를 사용하여 토큰화 및 표제어 추출, 불용어와 구두점 제거
    4. 전처리된 토큰을 하나의 문자열로 결합
    """
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc 
              if token.text not in stop_words and not token.is_stop and not token.is_punct and not token.is_space]
    return " ".join(tokens)

# KeyBERT 모델 로드
kw_model = KeyBERT()

# 1. JSON 파일 불러오기 (processed_all_reviews.json; app_id, name, review_text 컬럼 포함)
json_file = "processed_all_reviews.json"
df = pd.read_json(json_file, orient="records", encoding="utf-8")
print("불러온 데이터 (샘플):")
print(df.head())

# 2. 리뷰 텍스트 전처리 적용
df["cleaned_text"] = df["review_text"].apply(clean_review_text)
print("\n전처리 전/후 비교:")
print(df[["review_text", "cleaned_text"]].head())

# 3. KeyBERT를 이용해 각 리뷰에서 키워드 추출 (1~2그램, 상위 5개)
def extract_keywords(text, top_n=5):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), 
                                           stop_words='english', top_n=top_n)
    return keywords

df["keywords"] = df["cleaned_text"].apply(lambda x: extract_keywords(x, top_n=5))
print("\n키워드 추출 결과 (샘플):")
print(df[["review_text", "cleaned_text", "keywords"]].head())

# 4. 결과를 CSV 파일로 저장 (UTF-8-sig 인코딩)
output_csv = "review_keywords.csv"
df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"✅ 결과가 '{output_csv}' 파일로 저장되었습니다.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\02105\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\02105\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\02105\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


불러온 데이터 (샘플):
   app_id             name  review_id           timestamp  playtime_forever  \
0  428460  Kings of Israel  181973221 2024-12-05 18:14:39               292   
1  428460  Kings of Israel  181973221 2024-12-05 18:14:39               292   
2  605920           VOLTED  182479332 2024-12-11 08:09:54                49   
3  605920           VOLTED  186000812 2025-01-20 07:30:23               515   
4  605920           VOLTED  186000812 2025-01-20 07:30:23               515   

                                         review_text sentiment  
0                                                fun  Positive  
1                                                fun  Positive  
2  i have never played online for this game just ...  Positive  
3                         drive car shoot and defend  Positive  
4                         drive car shoot and defend  Positive  

전처리 전/후 비교:
                                         review_text  \
0                                                fun

In [9]:
import pandas as pd

# CSV 파일 읽기 (UTF-8-SIG 인코딩 사용)
csv_file = "review_keywords.csv"
df = pd.read_csv(csv_file, encoding="utf-8-sig")

# DataFrame을 JSON 파일로 변환하여 저장 (records 형식, force_ascii=False로 한글 보존, 인덴트 4)
json_file = "review_keywords.json"
df.to_json(json_file, orient="records", force_ascii=False, indent=4)

print(f"'{json_file}' 파일로 변환 완료.")


'review_keywords.json' 파일로 변환 완료.


In [1]:
import json
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# 1. 전처리된 리뷰 데이터를 불러오기 (JSON 파일)
with open('review_keywords.json', 'r', encoding='utf-8') as f:
    reviews = json.load(f)

# 2. 모든 리뷰의 텍스트를 하나의 문자열로 결합 (None 값 방지)
all_text = " ".join(item.get("cleaned_text", "") for item in reviews if item.get("cleaned_text"))

# 3. 불용어 집합을 정의하고 필요시 사용자 정의 불용어를 추가
stopwords = set(STOPWORDS)
custom_stopwords = ["game", "games", "play", "player", "good", "bad", "like", "one"]
stopwords.update(custom_stopwords)

# 4. 워드 클라우드 생성 함수
def generate_wordcloud(text, output_file, show_plot=True):
    if not text.strip():
        print(f"⚠️ 워드 클라우드 생성 실패: {output_file} (텍스트 없음)")
        return

    wordcloud = WordCloud(width=800, height=600, background_color="white",
                          stopwords=stopwords, max_words=200).generate(text)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.savefig(output_file)
    if show_plot:
        plt.show()  # 미리보기 출력
    plt.close()
    print(f"✅ 워드 클라우드 저장 완료: {output_file}")


# 5. 전체 리뷰 워드 클라우드 생성
generate_wordcloud(all_text, "wordcloud_all.png")

# 6. 감성별 텍스트 분리 및 워드 클라우드 생성
positive_text = " ".join(
    item.get("cleaned_text", "") 
    for item in reviews 
    if item.get("sentiment", "").lower() == "positive" and item.get("cleaned_text")
)
negative_text = " ".join(
    item.get("cleaned_text", "") 
    for item in reviews 
    if item.get("sentiment", "").lower() == "negative" and item.get("cleaned_text")
)

generate_wordcloud(positive_text, "wordcloud_positive.png")  # 긍정 리뷰 워드 클라우드
generate_wordcloud(negative_text, "wordcloud_negative.png")  # 부정 리뷰 워드 클라우드

# 7. 특정 게임(app_id)의 리뷰 워드 클라우드 생성
target_app_id = "1310990"  # 분석할 게임의 app_id (문자열로 지정)
game_reviews = [item.get("cleaned_text", "") 
                for item in reviews 
                if item.get("app_id") == target_app_id and item.get("cleaned_text")]
game_text = " ".join(game_reviews)

if game_reviews:
    generate_wordcloud(game_text, f"wordcloud_{target_app_id}.png")
else:
    print(f"⚠️ app_id {target_app_id}에 대한 리뷰 데이터가 없습니다.")

FileNotFoundError: [Errno 2] No such file or directory: 'review_keywords.json'